In [3]:
import time
stime = time.time()

import struct as st
import numpy as np

#opening file in readable binary mode
filename = {'images' : 'trainingSet/train-images-idx3-ubyte' ,'labels' : 'trainingSet/train-labels-idx1-ubyte'}
imagesfile = open(filename['images'],'rb')
labelsfile = open(filename['labels'],'rb')

#reading magic number
imagesfile.seek(0)
magic = st.unpack('>4B',imagesfile.read(4))

#reading dimensions of image dataset
nImg = st.unpack('>I',imagesfile.read(4))[0] #num of images
nR = st.unpack('>I',imagesfile.read(4))[0] #num of rows
nC = st.unpack('>I',imagesfile.read(4))[0] #num of column


images_array = np.array([])
labels_array = np.array([])

print(nImg,nR,nC)

#reading dimensions of image dataset
labelsfile.seek(8) #Since no. of items = no. of images and is already read
print("no. of images :: ",nImg)
print("no. of rows :: ",nR)
print("no. of columns :: ",nC)
temp_array = np.array([])
images10000_array = np.array([])
for i in range(1,nImg+1):
	#Read labels
	labels_array = np.append(labels_array,st.unpack('>B',labelsfile.read(1))[0])
	#Read training images
	if temp_array.size == 0:
		#invert the image as 255 is white and 0 is black
		temp_array = 255 - np.asarray(st.unpack('>784B',imagesfile.read(784))).reshape((nR,nC))
	else:
		nextimage = 255 - np.asarray(st.unpack('>784B',imagesfile.read(784))).reshape((nR,nC))
		if len(temp_array.shape)==2:
			temp_array = np.vstack((temp_array[None],nextimage[None]))
		else:
			temp_array = np.vstack((temp_array,nextimage[None]))
	
	#Extra stuffs to speed up the stacking process (took 51.804361105 seconds in my case)
	#Stacking each 1000 block to form a block of 10000
	if i%1000==0 and i != 0:
		if images10000_array.size == 0:
			images10000_array = temp_array
		else:
			images10000_array = np.vstack((images10000_array,temp_array))
		temp_array = np.array([])
		print("Time taken :: ",time.time()-stime)
	#Stacking each 10000 block to form the whole dataset
	if i%10000==0 and i != 0: 
		if images_array.size == 0:
			images_array = images10000_array
		else:
			images_array = np.vstack((images_array,images10000_array))
		images10000_array = np.array([])
		print((float(i)/nImg)*100,"% complete...")

print(labels_array.shape)
print(images_array.shape)

print("Time of execution : %s seconds" % str(time.time()-stime))

60000 28 28
no. of images ::  60000
no. of rows ::  28
no. of columns ::  28
Time taken ::  2.545091152191162
Time taken ::  3.4485082626342773
Time taken ::  4.304526329040527
Time taken ::  5.214833974838257
Time taken ::  6.089480638504028
Time taken ::  6.872518062591553
Time taken ::  7.641788721084595
Time taken ::  8.415234088897705
Time taken ::  9.222549438476562
Time taken ::  10.059660196304321
16.666666666666664 % complete...
Time taken ::  10.971642255783081
Time taken ::  11.847842931747437
Time taken ::  12.685598373413086
Time taken ::  13.544936180114746
Time taken ::  14.415477514266968
Time taken ::  15.224361419677734
Time taken ::  16.06997036933899
Time taken ::  16.84209108352661
Time taken ::  17.9439435005188
Time taken ::  18.845588207244873
33.33333333333333 % complete...
Time taken ::  21.256311893463135
Time taken ::  22.129063606262207
Time taken ::  22.886961936950684
Time taken ::  23.772178411483765
Time taken ::  24.66827917098999
Time taken ::  25.480

MemoryError: 

In [1]:
import time
stime = time.time()

import struct as st
import numpy as np

filename = {'images' : 'trainingSet/train-images-idx3-ubyte' ,'labels' : 'trainingSet/train-labels-idx1-ubyte'}

labels_array = np.array([])

data_types = {
        0x08: ('ubyte', 'B', 1),
        0x09: ('byte', 'b', 1),
        0x0B: ('>i2', 'h', 2),
        0x0C: ('>i4', 'i', 4),
        0x0D: ('>f4', 'f', 4),
        0x0E: ('>f8', 'd', 8)}

for name in filename.keys():
	if name == 'images':
		imagesfile = open(filename[name],'rb')
	if name == 'labels':
		labelsfile = open(filename[name],'rb')

#reading magic number
imagesfile.seek(0)
magic = st.unpack('>4B',imagesfile.read(4))

print(magic)
if(magic[0] and magic[1])or(magic[2] not in data_types):
    raise ValueError("File Format not correct")

nDim = magic[3]
print("Data is ",nDim,"-D")

#offset = 0004 for number of images
#offset = 0008 for number of rows
#offset = 0012 for number of columns
#32-bit integer (32 bits = 4 bytes)
imagesfile.seek(4)
nImg = st.unpack('>I',imagesfile.read(4))[0] #num of images/labels
nR = st.unpack('>I',imagesfile.read(4))[0] #num of rows
nC = st.unpack('>I',imagesfile.read(4))[0] #num of columns
nBytes = nImg*nR*nC
labelsfile.seek(8) #Since no. of items = no. of images and is already read
print("no. of images :: ",nImg)
print("no. of rows :: ",nR)
print("no. of columns :: ",nC)

#Read all data bytes at once and then reshape
images_array = 255 - np.asarray(st.unpack('>'+'B'*nBytes,imagesfile.read(nBytes))).reshape((nImg,nR*nC))
labels_array = np.asarray(st.unpack('>'+'B'*nImg,labelsfile.read(nImg))).reshape((nImg,1))

print(labels_array)
print(images_array)

print("Time of execution : %s seconds" % str(time.time()-stime))

(0, 0, 8, 3)
Data is  3 -D
no. of images ::  60000
no. of rows ::  28
no. of columns ::  28


MemoryError: 